# Sequence Detector

## Necessary modules

In [1]:
# Import Modules
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import sys
import tarfile
import seaborn
import random
from tensorflow.examples.tutorials.mnist import input_data
from IPython.display import display, Image
from scipy import ndimage
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

# Config the matlotlib backend as plotting inline in IPython
%matplotlib inline

## Generic functions

### Data Preprocessing Functions

In [2]:
last_percent_reported = None
num_classes = 10

def download_progress_hook(count, blockSize, totalSize):
  """A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 1% change in download progress.
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
        
def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  if force or not os.path.exists(filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

def maybe_extract(filename, force=False):
  """Extract a file (tar). If force=True, override the current files and folders"""
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('%s already present - Skipping extraction of %s.' % (root, filename))
  else:
    print('Extracting data for %s. This may take a while. Please wait.' % root)
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall()
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders

### Eval Functions

In [3]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == labels)
          / predictions.shape[0])

## 1. STEP ONE: SINGLE DIGIT MNIST DATA

### Downloading data

In [4]:
mnist = input_data.read_data_sets('../data/MNIST_data/', one_hot=False)
train_dataset = mnist.train.images
train_labels = mnist.train.labels
test_dataset = mnist.test.images
test_labels = mnist.test.labels
valid_dataset = mnist.validation.images
valid_labels = mnist.validation.labels
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Extracting ../data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/t10k-labels-idx1-ubyte.gz
Training set (55000, 784) (55000,)
Validation set (5000, 784) (5000,)
Test set (10000, 784) (10000,)


### Simple Logistic Regression

In [5]:
# Defining Params
IMAGE_SIZE = 28
NUM_CHANNELS = 1
NUM_LABELS = 10
BATCH_SIZE = 128

# Defining the Graph
graph = tf.Graph()

with graph.as_default():
  # Input init and constant init. According to the theory 
  tf_train_dataset = tf.placeholder(tf.float32, shape=(BATCH_SIZE, IMAGE_SIZE * IMAGE_SIZE)) 
  tf_train_labels = tf.placeholder(tf.int64, shape=(BATCH_SIZE)) 
  tf_valid_dataset = tf.constant(valid_dataset) 
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(tf.truncated_normal([IMAGE_SIZE * IMAGE_SIZE, NUM_LABELS])) # mean0,low std dev, random values
  biases = tf.Variable(tf.zeros([NUM_LABELS])) # zeros
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases # logistic regression
  loss = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss) # GD Optimizer
  
  # Predictions for the training, validation, and test data. (Probabilities)
  train_prediction = tf.nn.softmax(logits) 
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases) 
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
  
# Defining hyper-parameters
NUM_STEPS = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  for step in range(NUM_STEPS):
    offset = (step * BATCH_SIZE) % (train_labels.shape[0] - BATCH_SIZE)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + BATCH_SIZE), :]
    batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict) 
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print('Validation error: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Minibatch loss at step 0: 11.668404
Minibatch accuracy: 6.2%
Validation error: 11.8%
Minibatch loss at step 500: 0.735226
Minibatch accuracy: 81.2%
Validation error: 86.5%
Minibatch loss at step 1000: 0.380711
Minibatch accuracy: 89.8%
Validation error: 88.0%
Minibatch loss at step 1500: 0.717082
Minibatch accuracy: 80.5%
Validation error: 88.4%
Minibatch loss at step 2000: 0.502864
Minibatch accuracy: 84.4%
Validation error: 89.6%
Minibatch loss at step 2500: 0.301252
Minibatch accuracy: 91.4%
Validation error: 89.9%
Minibatch loss at step 3000: 0.049315
Minibatch accuracy: 99.2%
Validation error: 89.6%
Test accuracy: 89.1%


### 2-Layers NN

In [6]:
# Defining Params
IMAGE_SIZE = 28
NUM_CHANNELS = 1
NUM_LABELS = 10
BATCH_SIZE = 128
HIDDEN_NODES = 1024

# Defining the Graph
graph = tf.Graph()

with graph.as_default():
  # Input init and constant init. According to the theory 
  tf_train_dataset = tf.placeholder(tf.float32, shape=(BATCH_SIZE, IMAGE_SIZE * IMAGE_SIZE)) 
  tf_train_labels = tf.placeholder(tf.int64, shape=(BATCH_SIZE)) 
  tf_valid_dataset = tf.constant(valid_dataset) 
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(tf.truncated_normal([IMAGE_SIZE * IMAGE_SIZE, HIDDEN_NODES])) 
  b1 = tf.Variable(tf.zeros([HIDDEN_NODES])) 
  w2 = tf.Variable(tf.truncated_normal([HIDDEN_NODES, NUM_LABELS])) 
  b2 = tf.Variable(tf.zeros([NUM_LABELS])) 
  
  def twoNN(X):
      return tf.matmul(tf.nn.relu(tf.matmul(X, w1) + b1), w2) + b2
  
  # Training computation.
  logits = twoNN(tf_train_dataset) 
  
  loss = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss) # GD Optimizer
  
  # Predictions 
  train_prediction = tf.nn.softmax(logits) 
  valid_prediction = tf.nn.softmax(twoNN(tf_valid_dataset))
  test_prediction = tf.nn.softmax(twoNN(tf_test_dataset))
  
# Defining hyper-parameters
NUM_STEPS = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  for step in range(NUM_STEPS):
    offset = (step * BATCH_SIZE) % (train_labels.shape[0] - BATCH_SIZE)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + BATCH_SIZE), :]
    batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict) 
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print('Validation error: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Minibatch loss at step 0: 244.827911
Minibatch accuracy: 11.7%
Validation error: 24.2%
Minibatch loss at step 500: 0.713350
Minibatch accuracy: 93.0%
Validation error: 92.3%
Minibatch loss at step 1000: 0.008918
Minibatch accuracy: 99.2%
Validation error: 93.6%
Minibatch loss at step 1500: 0.266572
Minibatch accuracy: 93.8%
Validation error: 94.0%
Minibatch loss at step 2000: 0.229577
Minibatch accuracy: 96.1%
Validation error: 94.1%
Minibatch loss at step 2500: 0.022741
Minibatch accuracy: 99.2%
Validation error: 94.4%
Minibatch loss at step 3000: 0.000835
Minibatch accuracy: 100.0%
Validation error: 94.4%
Test accuracy: 93.6%


### CNN LeNet-5-like convolutional

In [7]:
IMAGE_SIZE = 28
NUM_CHANNELS = 1
NUM_LABELS = 10

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)).astype(np.float32)
  labels = labels.astype(np.int32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (55000, 28, 28, 1) (55000,)
Validation set (5000, 28, 28, 1) (5000,)
Test set (10000, 28, 28, 1) (10000,)


In [ ]:
BATCH_SIZE = 64
PATCH_SIZE = 5
DEPTH_1 = 32
DEPTH_2 = 64
HIDDEN_NODES = 256

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
  tf_train_labels = tf.placeholder(tf.int64, shape=(BATCH_SIZE))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # 5x5 Filter, depth 16 
  conv1_weights = tf.Variable(tf.truncated_normal([PATCH_SIZE, PATCH_SIZE, NUM_CHANNELS, DEPTH_1], stddev=0.1))
  conv1_biases = tf.Variable(tf.zeros([DEPTH_1]))
  
  # 5x5 Filter, depth 16
  conv2_weights = tf.Variable(tf.truncated_normal([PATCH_SIZE, PATCH_SIZE, DEPTH_1, DEPTH_2], stddev=0.1))
  conv2_biases = tf.Variable(tf.constant(1.0, shape=[DEPTH_2]))

  # Fully connected depth 256
  fc1_weights = tf.Variable(tf.truncated_normal([IMAGE_SIZE // 4 * IMAGE_SIZE // 4 * DEPTH_2, HIDDEN_NODES],
                                                   stddev=0.1))
  fc1_biases = tf.Variable(tf.constant(1.0, shape=[HIDDEN_NODES]))
  
  fc2_weights = tf.Variable(tf.truncated_normal([HIDDEN_NODES, NUM_LABELS], stddev=0.1))
  fc2_biases = tf.Variable(tf.constant(1.0, shape=[NUM_LABELS]))
  
  # Model.
  def model(data):
    
    # 2D Conv Layer
    conv = tf.nn.conv2d(data, conv1_weights, strides=[1, 1, 1, 1], padding='SAME')
    # Bias and rectified linear non-linearity.
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
    # Max pooling. The kernel size spec {ksize} also follows the layout of
    # the data. Here we have a pooling window of 2, and a stride of 2.
    pool = tf.nn.max_pool(relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # 2D Conv + ReLU + MaxPooling
    conv = tf.nn.conv2d(pool, conv2_weights, [1, 1, 1, 1], padding='SAME')
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv2_biases))
    pool = tf.nn.max_pool(relu, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    pool_shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3]])
        
    # Fully connected  
    hidden = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
    return tf.matmul(hidden, fc2_weights) + fc2_biases  
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))
  
NUM_STEPS = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(NUM_STEPS):
    offset = (step * BATCH_SIZE) % (train_labels.shape[0] - BATCH_SIZE)
    batch_data = train_dataset[offset:(offset + BATCH_SIZE), :, :, :]
    batch_labels = train_labels[offset:(offset + BATCH_SIZE)]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 10.894684
Minibatch accuracy: 10.9%
Validation accuracy: 11.3%
Minibatch loss at step 500: 0.403061
Minibatch accuracy: 89.1%
Validation accuracy: 91.6%
Minibatch loss at step 1000: 0.342183
Minibatch accuracy: 93.8%
Validation accuracy: 94.5%
Minibatch loss at step 1500: 0.201191
Minibatch accuracy: 95.3%
Validation accuracy: 96.1%
Minibatch loss at step 2000: 0.041332
Minibatch accuracy: 98.4%
Validation accuracy: 96.8%
